In [4]:
import os
import fitz  #fitz we'll extract text from PDF

To import fitz, we need PyMuPDF library

In [3]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.3 MB/s eta 0:00:00


In [82]:
#Extracting text from all pdf
def extract_text_from_pdf(folder_path):
  texts = []
  for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
      pdf_pdf = os.path.join(folder_path, filename)
      doc = fitz.open(pdf_pdf)
      text = " "
      for page in doc:
        text += page.get_text("text")
  return text

#Extracting text from all pdf
def extract_text_from_pdf(folder_path):
  texts = []
  for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
      pdf_pdf = os.path.join(folder_path, filename)
      doc = fitz.open(pdf_path)
      text = " "
      for page in doc:
        text += page.get_text(text)
  return texts

pdf_folder = ["Rohith_CV.pdf","budget_speech"]
document = extract_text_from_pdf("demoo/")

Problems in your code:

Variable name mismatch:
You wrote pdf_pdf = ... but then tried fitz.open(pdf_path) → should be consistent.

Wrong get_text() usage:
page.get_text("text") is correct (not page.get_text(text)).

Not appending text to texts list:
Right now your function never returns any extracted content.

pdf_folder usage:
You set pdf_folder = ["Rohith_CV.pdf","budget_speech"], but the function expects a folder path (string, not list).

Fixed code:

import os
import fitz  # PyMuPDF

# Extract text from all PDFs in a folder
def extract_text_from_pdf(folder_path):
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            doc = fitz.open(pdf_path)
            text = ""
            for page in doc:
                text += page.get_text("text")  # extract page text
            texts.append(text)
    return texts

# Example usage
pdf_folder = "demoo/"   # this should be a folder containing PDFs
documents = extract_text_from_pdf(pdf_folder)

print("Extracted", len(documents), "PDFs")
print(documents[0][:500])  # print first 500 chars of first PDF

In [83]:
pdf_folder = ("demoo/")

In [84]:
pdf_folder

'demoo/'

In [85]:
document = extract_text_from_pdf(pdf_folder)

In [88]:
document[2]

'O'

In [80]:
print("Extracted", len(document), "PDFsTotalLength")

Extracted 92657 PDFsTotalLength


In [91]:
print(document[1][:250])

G


In [61]:
pip install pytesseract pdf2image pillow

In [62]:
from pdf2image import convert_from_path
import pytesseract

In [63]:
def extract_text_with_ocr(pdf_path):
  pages = convert_from_path(pdf_path)
  text = " "
  for page in pages:
    text+=pytesseract.image_to_string(page)
  return text

In [72]:
pdf_path = "demoo/Rohith_CV.pdf"

In [73]:
ocr_text = extract_text_with_ocr(pdf_path)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [92]:
doc = fitz.open("demoo/Rohith_CV.pdf")
text = ""
for page in doc:
  text += page.get_text("text")
  if not text.strip():
    text += page.get_text("ocr")
print(text[:500])

ROHITH GOWDA K 
 
Mobile Number: 8431887589                                                                                Gmail: rohithgowdak77@gmail.com 
                                                                                                                                                               
 
Summary: 
 
➢ An aspiring DevOps Engineer with hands-on experience deploying, automating, and optimizing software 
development processes. Eager to contribute to a dynamic team and gr


In [94]:
def extract_text_auto(pdf_path):
  doc = fitz.open(pdf_path)
  text=""
  ocr_needed = False

  for page in doc:
    page_text = page.get_text("text")
    if page_text.strip():
      text += page_text
    else:
      ocr_needed = True

  doc.close()

  if ocr_needed:
    print(f"Using OCR for: {pdf_path}")
    pages = convert_from_path(pdf_path)
    for img_pages in pages:
      text+=pytesseract.image_to_string(img_pages)

  return text


In [123]:
folder = "demoo"
all_text = {}

In [96]:
for file in os.listdir(folder):
  if file.endswith(".pdf"):
    pdf_path = os.path.join(folder, file)
    extracted_text = extract_text_auto(pdf_path)
    all_text[file] = extracted_text
    print(f"Extracted {len(extracted_text)} characters from {file}")

Extracted 3114 characters from Rohith_CV.pdf
Extracted 4940 characters from sk.pdf
Using OCR for: demoo/budget_speech.pdf


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [124]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path

def extract_text_auto(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    # Try extracting text normally from all pages
    for page in doc:
        page_text = page.get_text("text")
        if page_text.strip():
            text += page_text

    doc.close()

    # If no text found at all → fallback to OCR
    if not text.strip():
        print(f"⚡ Using OCR for: {pdf_path}")
        pages = convert_from_path(pdf_path)
        for img_page in pages:
            text += pytesseract.image_to_string(img_page)

    return text


In [125]:
folder = "demoo"
all_text = {}

In [126]:
for file in os.listdir(folder):
  if file.endswith(".pdf"):
    pdf_path = os.path.join(folder, file)
    extracted_text = extract_text_auto(pdf_path)
    all_text[file] = extracted_text
    print(f"Extracted {len(extracted_text)} characters from {file}")

Extracted 3114 characters from Rohith_CV.pdf
Extracted 4940 characters from sk.pdf
Extracted 92656 characters from budget_speech.pdf


In [127]:
#Now save results into .txt
for fname, content in all_text.items():
  with open(fname.replace(".pdf",".txt"),"w",encoding="utf-8") as f:f.write(content)


In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [103]:
documents = list(all_text.values())

In [104]:
processed_docs = [doc.lower().strip() for doc in documents]

In [113]:
vectorizer = TfidfVectorizer()

In [114]:
X = vectorizer.fit_transform(processed_docs)

In [115]:
print("shape of TF-IDF matrix:",X.shape)

shape of TF-IDF matrix: (3, 2875)


In [116]:
print(vectorizer.get_feature_names_out()[:50])

['00' '000' '001' '01' '02' '04' '07' '10' '100' '1000' '101' '102' '103'
 '104' '105' '106' '107' '108' '109' '10d' '10mm' '11' '110' '111' '111a'
 '112' '112a' '113' '114' '115' '115ub' '115vp' '116' '117' '118' '119'
 '12' '120' '121' '122' '122b' '123' '124' '125' '126' '127' '127a' '127b'
 '127c' '127d']


In [108]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(X)
print(similarities)

[[1.         0.14293748 0.33214492]
 [0.14293748 1.         0.39450392]
 [0.33214492 0.39450392 1.        ]]


In [118]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    token_pattern=r"[a-zA-Z]{2,}",
    max_features=5000
)

X = vectorizer.fit_transform(processed_docs)

In [119]:
print(vectorizer.get_feature_names_out()[:50])

['aa' 'aab' 'aatmanirbharta' 'ab' 'abilities' 'ability' 'able' 'academic'
 'accelerate' 'accelerating' 'accepting' 'access' 'accordingly' 'account'
 'accounts' 'accrued' 'accruing' 'accumulated' 'achieve' 'achieving'
 'acid' 'act' 'action' 'active' 'activities' 'actually' 'adaptability'
 'add' 'added' 'adding' 'addition' 'additional' 'address' 'addressing'
 'adjust' 'adjustments' 'administration' 'admirable' 'adolescent' 'adopt'
 'advance' 'affordability' 'affordable' 'aforesaid' 'age' 'agencies'
 'aggregate' 'ago' 'agreed' 'agreements']


In [130]:
filenames = list(all_text.keys())   # same order as documents

In [131]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()

def top_keywords_for_doc(doc_index, top_n=10):
    row = X[doc_index].toarray().flatten()   # TF-IDF values for that doc
    top_indices = row.argsort()[::-1][:top_n]
    keywords = [(feature_names[i], row[i]) for i in top_indices]
    return keywords

# Example: top keywords for each file
for i, fname in enumerate(filenames):
    print(f"\n {fname}")
    print(top_keywords_for_doc(i, top_n=10))



 Rohith_CV.pdf
[('tools', np.float64(0.37166323261457107)), ('jenkins', np.float64(0.20647957367476172)), ('rest', np.float64(0.20647957367476172)), ('experience', np.float64(0.20647957367476172)), ('terraform', np.float64(0.20647957367476172)), ('tool', np.float64(0.16518365893980938)), ('docker', np.float64(0.16518365893980938)), ('code', np.float64(0.16518365893980938)), ('aws', np.float64(0.16518365893980938)), ('instance', np.float64(0.16518365893980938))]

 sk.pdf
[('plot', np.float64(0.3657262332156193)), ('test', np.float64(0.3454081091480849)), ('train', np.float64(0.30477186101301607)), ('clf', np.float64(0.22349936474287843)), ('sklearn', np.float64(0.22349936474287843)), ('estimator', np.float64(0.20318124067534407)), ('predictions', np.float64(0.18286311660780966)), ('metrics', np.float64(0.16254499254027524)), ('values', np.float64(0.16254499254027524)), ('display', np.float64(0.1545245706221628))]

 budget_speech.pdf
[('section', np.float64(0.3078271362281662)), ('tax',

In [134]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(X)

# Example: compare first doc with others
print("Similarity of first doc vs all:")
for i, fname in enumerate(filenames):
    print(f"{filenames[2]} vs {fname}: {similarities[2][i]:.3f}")


Similarity of first doc vs all:
budget_speech.pdf vs Rohith_CV.pdf: 0.031
budget_speech.pdf vs sk.pdf: 0.017
budget_speech.pdf vs budget_speech.pdf: 1.000


In [135]:
query = "machine learning"
query_vec = vectorizer.transform([query])

similarities = cosine_similarity(query_vec, X).flatten()

# Sort by relevance
results = sorted(zip(filenames, similarities), key=lambda x: -x[1])

print("\n🔎 Search results for:", query)
for fname, score in results:
    print(f"{fname}: {score:.3f}")



🔎 Search results for: machine learning
sk.pdf: 0.029
Rohith_CV.pdf: 0.000
budget_speech.pdf: 0.000


In [137]:
print("machine" in vectorizer.get_feature_names_out())
print("learning" in vectorizer.get_feature_names_out())
print("machine learning" in all_text["Rohith_CV.pdf"].lower())

True
True
False
